<a href="https://colab.research.google.com/github/mfvalle/Maths_for_MachineLearning/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrega 1

Sergio Quiroga Sandoval

# Reproducibility
we will provide a reproducible notebook with automatic download data from a github repository created for this project



Important: run the following command

In [ ]:
!ls
!git clone https://github.com/SergioQS/Machine-Learning.git  

Machine-Learning  sample_data
fatal: destination path 'Machine-Learning' already exists and is not an empty directory.


If you see sample_data then you can run the following code ONLY ONCE


In [ ]:

import pandas as pd
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import math
from sklearn.metrics import confusion_matrix

def hms_to_s(s):                # funcion para pasar horas:minutos:segundos a minutos
    t = 0
    for u in s.split(':'):
        t = 60 * t + int(u)
    t = t/60
    return t

## Tasks
for both problems

1) Specify which Machine Learning problem are you solving

2) Provide a short summary of the features and the labels you are working on

3) Please answer the following questions: 

    a) Are these datasets linearly separable?
    b) Are these datasets randomly chosen 
    c) The sample size is enough to guarantee generalization

4) Provide an explanation how and why the code is working

5) Show some examples to illustrate that the method is working properly............  (evaluarlo para ejemplos del data test)

6) Provide quantitative evidence for generalization using the provided dataset.....

## First dataset: occupancy rate

### 1) Machine Learning problem 

We are solving a binary classification problem using support vector machines, first we have a dataset consisting of 
7 dimensional vectors with the information of "date","Temperature","Humidity","Light","CO2","HumidityRatio","Occupancy"

The goal is implementing a support vector machines model for predicting the "Occupancy" given the other 6 data.
For this we will check if the dataset is linearly separable, for correctly implementing support vector machines.

### 2) short summary of the features and the labels

Our features and format are the following



*   "date": date time year-month-day hour:minute:second
*   "Light": (light in lux)


"Temperature": in celcius
"humidity": (Relative Humidity, %)
"co2":(CO2, in ppm)
"HumidityRatio":(Derived quantity from temperature and relative humidity, in kgwater-vapor/kg-air)


lets see if there is a relation between each feature and the "Occupancy". 
it is important because we need to be sure we use adecuate features on the dataset for predicting occupancy. 

"date": important data because it can be related with the normal values of "Temperature","Humidity" and "Light",
because these may vary during the day so we will consider for our model the hour in the day where the 7 data was recorded.
"Temperature": temperatue is related with occupancy because humans breath can increase temperature in a room.

"Humidity": humidity increases with the number of persons in a room, so it is related with occupancy.

"Light" : an occupied room may have the light turned on".

"CO2":  the levels of CO2 increase with the more people there are in a room.

"HumidityRatio": 

Occupancy, 0 or 1, 0 for not occupied, 1 for occupied status

### 3) a. Are these datasets linearly separable?
for checking data is linearly separable, lets see if an svm can overfit perfectly the dataset

In [ ]:
# importing data
datatraining_text = pd.read_csv('/content/Machine-Learning/datatraining.csv', encoding='utf-8')

print(datatraining_text.head)
print(type(datatraining_text))

<bound method NDFrame.head of      "date","Temperature","Humidity","Light","CO2","HumidityRatio","Occupancy"
0     "1","2015-02-04 17:51:00",23.18,27.272,426,721...                       
1     "2","2015-02-04 17:51:59",23.15,27.2675,429.5,...                       
2     "3","2015-02-04 17:53:00",23.15,27.245,426,713...                       
3     "4","2015-02-04 17:54:00",23.15,27.2,426,708.2...                       
4     "5","2015-02-04 17:55:00",23.1,27.2,426,704.5,...                       
...                                                 ...                       
8138  "8139","2015-02-10 09:29:00",21.05,36.0975,433...                       
8139  "8140","2015-02-10 09:29:59",21.05,35.995,433,...                       
8140  "8141","2015-02-10 09:30:59",21.1,36.095,433,7...                       
8141  "8142","2015-02-10 09:32:00",21.1,36.26,433,82...                       
8142  "8143","2015-02-10 09:33:00",21.1,36.2,447,821...                       

[8143 rows x 1 column

In [ ]:
# preparing the dataset 
data_dict = np.array(datatraining_text)
for i in range(len(data_dict)):
    data_dict[i][0] = (data_dict[i][0].split(","))


class0 = []
class1 = []
fechas = []

"""ingeniería de características. usando solo la hora del día del dato de la fecha, porque puede tener incidencia en niveles de humedad base 
durante el dia"""

for i in range(len(data_dict)):
    del data_dict[i][0][0]                    # borrar la numeración
    fechas.append(data_dict[i][0][0])         # crear un array con la fecha de cada vector de datos
    fechas[i] = fechas[i].split( )            # manejo y limpieza de la fecha
    del fechas[i][0]
    fechas[i] = ''.join(fechas[i])
    fechas[i] = fechas[i].replace('"', '')
    fechas[i] = hms_to_s(fechas[i])
    data_dict[i][0][0] = fechas[i]                          # reincorporar hora convertida solo a minutos en el diccionario
    data_dict[i][0] = [float(x) for x in data_dict[i][0]]   # convertir las entradas de cada array a float type
    if data_dict[i][0][6] == 0:                             # separar el dataset en las dos clases posibles dependiendo del output
        class0.append(data_dict[i][0])
    else:
        class1.append(data_dict[i][0])

for i in range(len(class0)):
   del class0[i][-1]
for i in range(len(class1)):
   del class1[i][-1]

print(class1[0:10])

""" el dataset está separado en dos clases, ahora hay que crear un vector de etiquetas de la forma y = [1,1,1,1,...,1,-1,-1,-1,...,-1] 
 donde el numero de 1 sea len(class0) y el numero de -1 sea len(class1) y luego unir las dos clases en un solo vector de 
 caracteríaticas (X = [...]) (esto se hace para poder meter el dataset en sklearn.svm)
"""
y = []

print("lenghts of the classes are: ", len(class0),"and: ", len(class1))

for i in range(len(class0)):
    y.append(1)
for i in range(len(class1)):
    y.append(-1)


X = class0 + class1


y = np.array(y)
X = np.array(X)

print( "lenght of X is: ", len(X))  

print( "lenght of y is: ", len(y))   # X y y ya tienen la misma longitud.


# implementación del svm:

"""
para implementar la funcion clf.fit(X, y) los datos deben tener esta forma:
X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = np.array([1, 1, 2, 2])
"""
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X, y)

predict_X = clf.predict(X)

#if predict_X.all() == y.all():
  
if np.array_equiv(predict_X, y):
    print("el modelo predice perfectamente todos los x del entrenamiento (overfit), por lo tanto el dataset es linealmente separable")
else:
    print("el modelo no predice bien algun/algunos ejemplos del dataset de training")

print((predict_X == y).all(),)

#el modelo se está equivocando en los siguientes datos del training set. 
indices_dif = []
for i in range(len(y)):
  if predict_X[i] != y[i]:
    indices_dif.append(i)

print("los siguientes son los indices de los elementos con predicción erronea",indices_dif)
#print("la matriz de confusion se ve de la siguiente forma (ya normalizada con el valor true): \n", confusion_matrix(y, predict_X,normalize="true"))

[[1071.0, 23.18, 27.272, 426.0, 721.25, 0.00479298817650529], [1071.9833333333333, 23.15, 27.2675, 429.5, 714.0, 0.00478344094931065], [1073.0, 23.15, 27.245, 426.0, 713.5, 0.00477946352442199], [1074.0, 23.15, 27.2, 426.0, 708.25, 0.00477150882608175], [1075.0, 23.1, 27.2, 426.0, 704.5, 0.00475699293331518], [1075.9833333333333, 23.1, 27.2, 419.0, 701.0, 0.00475699293331518], [1077.0, 23.1, 27.2, 419.0, 701.666666666667, 0.00475699293331518], [1077.9833333333333, 23.1, 27.2, 419.0, 699.0, 0.00475699293331518], [1078.9833333333333, 23.1, 27.2, 419.0, 689.333333333333, 0.00475699293331518], [1080.0, 23.075, 27.175, 419.0, 688.0, 0.00474535071966655]]
lenghts of the classes are:  6414 and:  1729
lenght of X is:  8143
lenght of y is:  8143
el modelo no predice bien algun/algunos ejemplos del dataset de training
False
los siguientes son los indices de los elementos con predicción erronea [811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 83

### 3) b. Are these datasets randomly chosen and 
 




We need to make an analysis of the possible ways in wich the data was collected.

since data is from occupancy of rooms it is possible that there are some biases in the collecting of data like the following ones:
*   the season in wich data was recorded.
*   







### 3) c. The sample size is enough to guarantee generalization.

Taking into account the given data of 8143 and that there are 6 features per data. 
We have a 6 dimensional parameter space, so we see that the amount of available data is larger than ten times the dimensions of the dataset.
for that reason we could say that there is enough data to generalize

### 4) Provide an explanation how and why the code is working

the initial code wasnt working because:

*   the directionalvectors were smaller than expected by the size of the data in occupancy dataset.
*   also it was designed for a two dimensional problem, so it had plots implemented for visualising the dataset.
*   the step size was not optimum.
*   the algorithm was not reading correctly the data.







For this reason we used the scikit implementation of SVM.

### 5) Show some examples to illustrate that the method is working properly.

In [ ]:
print("lets see some examples of the model working: ", clf.predict(X[0:5]), y[0:5])

print(clf.predict(X[60:65]), y[60:65])

print(clf.predict(X[4060:4065]), y[4060:4065])

print(clf.predict(X[7060:7065]), y[7060:7065])


print(clf.predict(X[6060:6065]), y[6060:6065])


print(clf.predict(X[8060:8065]), y[8060:8065])

lets see some examples of the model working:  [1 1 1 1 1] [1 1 1 1 1]
[1 1 1 1 1] [1 1 1 1 1]
[1 1 1 1 1] [1 1 1 1 1]
[-1 -1 -1 -1 -1] [-1 -1 -1 -1 -1]
[1 1 1 1 1] [1 1 1 1 1]
[-1 -1 -1 -1 -1] [-1 -1 -1 -1 -1]


### 6) Provide quantitative evidence for generalization using the provided dataset

In [ ]:
# punto 6
# veamos si el modelo generaliza  
""" lo que haremos es separar el dataset en dos conjuntos x_test, x_train, y_test, y_train 
luego entrenamos el modelo con el conjunto train y lo evaluamos en los ejemplos del conjunto test
"""
data_test = pd.read_csv("/content/Machine-Learning/datatest.csv", encoding='utf-8') 
# le haremos el mismo proceso anterior al datatest

data_dict_test = np.array(data_test)


for i in range(len(data_dict_test)):
    data_dict_test[i][0] = (data_dict_test[i][0].split(","))


class0_test = []
class1_test = []
#ingeniería de características. usando solo la hora del día del dato de la fecha, porque puede tener incidencia en niveles de humedad base
fechas_test = []

for i in range(len(data_dict_test)):
    del data_dict_test[i][0][0]                    # borrar la numeración
    fechas_test.append(data_dict_test[i][0][0])         #crear un array con la fecha de cada vector de datos
    fechas_test[i] = fechas_test[i].split( )            # manejo y limpieza de la fecha
    del fechas_test[i][0]
    fechas_test[i] = ''.join(fechas_test[i])
    fechas_test[i] = fechas_test[i].replace('"', '')
    fechas_test[i] = hms_to_s(fechas_test[i])
    data_dict_test[i][0][0] = fechas_test[i]            # reincorporar hora convertida solo a minutos en el diccionario

    if data_dict_test[i][0][6] == "0":             # separar el dataset en las dos clases posibles dependiendo del output
        class0_test.append(data_dict_test[i][0])
    else:
        class1_test.append(data_dict_test[i][0])

for i in range(len(class0_test)):
   del class0_test[i][-1]
for i in range(len(class1_test)):
   del class1_test[i][-1]
for i in range(len(class0_test)):
    for j in range(len(class0_test[i])):
        class0_test[i][j] = float(class0_test[i][j])
for i in range(len(class1_test)):
    for j in range(len(class1_test[i])):
        class1_test[i][j] = float(class1_test[i][j])

#print(class1_test[0:10])

y_test = []
for i in range(len(class0_test)):
    y_test.append(1)
for i in range(len(class1_test)):
    y_test.append(-1)
X_test = class0_test + class1_test
y_test = np.array(y_test)
X_test = np.array(X_test)

print( "lenght of X_test is: ", len(X_test))  
print( "lenght of y_test is: ", len(y_test))   

#entrenando el modelo:
#clf.fit(x_train, y_train) ya se corrió antes
predict = clf.predict(X_test)

indices_diferentes = []
n=0
while n < len(predict): 
    if predict[n] != y_test[n]:
        indices_diferentes.append(n)
    n = n+1
print("el modelo se equivoca en los siguientes indices de datos", indices_diferentes)

print('Accuracy of the model in the Test set is:', clf.score(X_test, y_test))

#-----------------
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_test, predict)
recall = recall_score(y_test, predict)
f1 = f1_score(y_test, predict)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
#-----------------
if recall_score(y_test, predict) > 0.95:
    print("luego el modelo es capaz de generalizar en datos no vistos anteriormente, pues la clase class_0_test tiene: ", len(class0_test), " elementos y class_1_test tiene: ", len(class1_test), "por lo que es adecuado hablar de recall")


#print('Predicted Values from Classifier:', predict)
#print('Actual Output is:', y_test)


lenght of X_test is:  2665
lenght of y_test is:  2665
el modelo se equivoca en los siguientes indices de datos [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 866, 867, 868, 869, 870, 871, 1677, 1678, 1679, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 1690, 1691, 1692, 1896, 2601, 2602]
Accuracy of the model in the Test set is: 0.9778611632270169
Precision: 0.9981707317073171
Recall: 0.9669226225634967
F1 Score: 0.9822982298229822
luego el modelo es capaz de generalizar en datos no vistos anteriormente, pues la clase class_0_test tiene:  1693  elementos y class_1_test tiene:  972 por lo que es adecuado hablar de recall


In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_test, y_test, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.96 accuracy with a standard deviation of 0.04


## Second Dataset: Banknote Autentication

### 1) Machine Learning problem 

The problem we are solving is creating a binary classification model into two classes (real, counterfeited) for banknote authentication using Support Vector Machines.


Data were extracted from images that were taken from genuine and forged banknote-like specimens. For digitization, an industrial camera usually used for print inspection was used. The final images have 400x 400 pixels. Due to the object lens and distance to the investigated object gray-scale pictures with a resolution of about 660 dpi were gained. Wavelet Transform tool were used to extract features from images.



### 2) short summary of the features and the labels

The dataset has vector parameters with 4 dimensions and the class corresponding to them.

The first 3 parameters refer to the corresponding moments of the wavelet transformed image.
1. variance of Wavelet Transformed image (continuous)
2. skewness of Wavelet Transformed image (continuous)
3. curtosis of Wavelet Transformed image (continuous)
4. entropy of image (continuous)
5. class (integer)

### 3) a. Are these datasets linearly separable?

For checking linear separability we will try to completely overfit the dataset using a hard margin svm implementation

In [ ]:

datatraining_text2 = pd.read_csv('/content/Machine-Learning/data_banknote_authentication.csv', encoding='utf-8')

print(datatraining_text2.head)
print(type(datatraining_text2))

<bound method NDFrame.head of         3.6216,8.6661,-2.8073,-0.44699,0
0        4.5459,8.1674,-2.4586,-1.4621,0
1         3.866,-2.6383,1.9242,0.10645,0
2        3.4566,9.5228,-4.0112,-3.5944,0
3       0.32924,-4.4552,4.5718,-0.9888,0
4        4.3684,9.6718,-3.9606,-3.1625,0
...                                  ...
1366   0.40614,1.3492,-1.4501,-0.55949,1
1367    -1.3887,-4.8773,6.4774,0.34179,1
1368  -3.7503,-13.4586,17.5932,-2.7771,1
1369    -3.5637,-8.3827,12.393,-1.2823,1
1370    -2.5419,-0.65804,2.6842,1.1952,1

[1371 rows x 1 columns]>
<class 'pandas.core.frame.DataFrame'>


In [ ]:


data_dict2 = np.array(datatraining_text2)
for i in range(len(data_dict2)):
    data_dict2[i][0] = (data_dict2[i][0].split(","))


class1_2 = []
class0_2 = []
for i in range(len(data_dict2)):   
    data_dict2[i][0] = [float(x) for x in data_dict2[i][0]]              
    if data_dict2[i][0][4] == 0:             # separar el dataset en las dos clases posibles dependiendo del output
        data_dict2[i][0] = [float(x) for x in data_dict2[i][0]]
        class0_2.append(data_dict2[i][0])
    else:
        class1_2.append(data_dict2[i][0])

for i in range(len(class0_2)):
   del class0_2[i][-1]
for i in range(len(class1_2)):
   del class1_2[i][-1]


#print(class1_2[0:10])
#print(class0_2[0:10])
y_2 = []

print("lenghts of the classes are: ", len(class0_2),"and: ", len(class1_2))

for i in range(len(class0_2)):
    y_2.append(1)
for i in range(len(class1_2)):
    y_2.append(-1)


X_2 = class0_2 + class1_2


y_2 = np.array(y_2)
X_2 = np.array(X_2)

print( "lenght of X is: ", len(X_2))  

print( "lenght of y is: ", len(y_2)) 


clf_2 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf_2.fit(X_2, y_2)

predict_X_2 = clf_2.predict(X_2)



if np.array_equiv(predict_X_2, y_2):
    print("el modelo predice perfectamente todos los x del entrenamiento (overfit), por lo tanto el dataset es linealmente separable")
else:
    print("el modelo no predice bien algun/algunos ejemplos del dataset")

print((predict_X_2 == y_2).all(),)

indices_dif_2 = []
for i in range(len(y_2)):
  if predict_X_2[i] != y_2[i]:
    indices_dif_2.append(i)

print("los siguientes son los indices de los elementos con predicción erronea",indices_dif_2)

lenghts of the classes are:  761 and:  610
lenght of X is:  1371
lenght of y is:  1371
el modelo predice perfectamente todos los x del entrenamiento (overfit), por lo tanto el dataset es linealmente separable
True
los siguientes son los indices de los elementos con predicción erronea []


### 3) b. Are these datasets randomly chosen 

these are some of the possible biases during the collection of data

*   The counterfeited bills may have been from the same forger and it is possible that the model will not generalize to other forger.
*   The genuine bills may be in a bad state so they may be misclassified.


So the collection of data is not guaranteed to be randomly chosen.




### 3) c. The sample size is enough to guarantee generalization.

To guarantee generalization we need at least around 10 times the VC dimension, so it would me around 500 data, our dataset contains 1371 data, so we could say it guarantee generalization.


### 4) Provide an explanation how and why the code is working

the initial code wasnt working because:

*   the directionalvectors were smaller than expected by the size of the data in occupancy dataset.
*   also it was designed for a two dimensional problem, so it had plots implemented for visualising the dataset.
*   the step size was not optimum.
*   the algorithm was not reading correctly the data.







For this reason we used the scikit implementation of SVM.

### 6) Provide quantitative evidence for generalization using the provided dataset

as a first attempt to have an insight on the generalization of the model we will make a 60/40 partition for train/test in the original dataset.
then in the next block of code we will implement a cross validation testing

In [ ]:
print(X_2[0:3])
print(y_2[0:3])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_2, y_2, test_size=0.4, random_state=0)


clf_general = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf_general.fit(X_train, y_train)

predict_X_test = clf_general.predict(X_test)


print("the score of the model in testing is: ",clf_general.score(X_test, y_test))


errors = []
for i in range(len(y_test)):
  if predict_X_test[i] != y_test[i]:
    errors.append(i)
print("los siguientes son los indices de los elementos con predicción erronea en el testing: ", errors)
print("el siguiente valor de verdad indica si la prediccion es exactamente igual al y_test: ",(predict_X_test == y_test).all())

[[ 4.5459   8.1674  -2.4586  -1.4621 ]
 [ 3.866   -2.6383   1.9242   0.10645]
 [ 3.4566   9.5228  -4.0112  -3.5944 ]]
[1 1 1]
the score of the model in testing is:  1.0
los siguientes son los indices de los elementos con predicción erronea en el testing:  []
el siguiente valor de verdad indica si la prediccion es exactamente igual al y_test:  True


take a look on the following lines of scikit docs: 
"When evaluating different settings (“hyperparameters”) for estimators, such as the C setting that must be manually set for an SVM, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set."

this is happening, so we will implement a cross validation 

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf_general, X_test, y_test, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_test, predict_X_test)
recall = recall_score(y_test, predict_X_test)
f1 = f1_score(y_test, predict_X_test)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

0.99 accuracy with a standard deviation of 0.01
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


### 5) Show some examples to illustrate that the method is working properly.

In [ ]:
print("lets see some examples of the model working: ", (predict_X_test[0:5]), y_test[0:5])

print((predict_X_test[60:65]), y_test[60:65])
print((predict_X_test[500:505]), y_test[500:505])

lets see some examples of the model working:  [-1  1 -1  1  1] [-1  1 -1  1  1]
[-1  1  1  1  1] [-1  1  1  1  1]
[ 1  1 -1 -1  1] [ 1  1 -1 -1  1]
